<h1 style="text-align: center; font-size: 50px;">Vanilla RAG Chatbot with LangChain</h1>

Retrieval-Augmented Generation (RAG) is an architectural approach that can enhance the effectiveness of large language model (LLM) applications using customized data. In this example, we use LangChain, an orchestrator for language pipelines, to build an assistant capable of loading information from a web page and use it for answering user questions.

# Notebook Overview
- Start Execution
- Install and Import Libraries
- Configure Settings
- Verify Assets
- Data Loading
- Creation of Chunks
- Retrieval
- Model Setup
- Chain Creation
- Testing the RAG Chain

# Start Execution

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("run_workflow_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()  

logger.info("Notebook execution started.")

2025-09-18 12:12:20 - INFO - Notebook execution started.


# Install and Import Libraries

By using our Local GenAI workspace image, many of the necessary libraries to work with RAG already come pre-installed - in our case, we just need to add the connector to work with PDF documents

In [3]:
%%time

%pip install -r ../requirements.txt --quiet

Note: you may need to restart the kernel to use updated packages.
CPU times: user 62.2 ms, sys: 18.2 ms, total: 80.5 ms
Wall time: 4.46 s


In [4]:
# === Standard Library Imports ===
import os
import sys
import warnings
from datetime import datetime
from pathlib import Path
from typing import Dict, Any, Optional, Union, List, Tuple

# === Third-Party Imports ===
import torch
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline, HuggingFaceEndpoint
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableMap
from langchain.schema.document import Document
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Define the relative path to the 'src' directory (one level up from current working directory)
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# === Project-Specific Imports (from src.utils) ===
from src.utils import (
    load_secrets,
    load_config,
    load_secrets_to_env,
    configure_proxy,
    initialize_llm,
    configure_hf_cache,
    get_context_window, 
    dynamic_retriever, 
    format_docs_with_adaptive_context,
    login_huggingface
)
from src.prompt_templates import format_rag_chatbot_prompt

# === Import the service class ===
from core.chatbot_service.chatbot_service import ChatbotService

/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
/opt/conda/lib/python3.12/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


# Configure Settings

In [5]:
warnings.filterwarnings("ignore")

In [6]:
CONFIG_PATH = "../configs/config.yaml"
SECRETS_PATH = "../configs/secrets.yaml"
DATA_PATH = "../data"
LOCAL_MODEL_PATH = "/home/jovyan/datafabric/meta-llama3.1-8b-Q8/Meta-Llama-3.1-8B-Instruct-Q8_0.gguf"
DEMO_FOLDER = "../demo"

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Configuration and Secrets Loading

In this section, we load configuration parameters and API keys from separate YAML files. This separation helps maintain security by keeping sensitive information (API keys) separate from configuration settings.

- **config.yaml**: Contains non-sensitive configuration parameters like model sources and URLs
- **secrets.yaml**: Contains sensitive API keys for services like HuggingFace
- *(Optional for Premium users)* Secrets such as API keys for services like HuggingFace can be stored as environment variables for the project and loaded into the notebook (see the project's README file for steps on how to save secrets in Secrets Manager).

In [8]:
# Configuration parameters for the RAG chatbot
MODEL_SOURCE = "local"  # Can be: "local", "hugging-face-local", "hugging-face-cloud"
CHATBOT_SERVICE_NAME = "VanillaRAGChatbot"

# Load secrets from secrets.yaml file (if it exists) into environment
if Path(SECRETS_PATH).exists():
    load_secrets_to_env(SECRETS_PATH)
else:
    print(f"No secrets file found at {SECRETS_PATH}; relying on preexisting environment")

# Retrieve secrets from environment
try:
    secrets = load_secrets()
except ValueError:
    secrets = {}

# Load configuration and secrets
config = load_config(CONFIG_PATH)

print("✅ Configuration loaded successfully")
print("✅ Secrets loaded successfully")

No secrets file found at ../configs/secrets.yaml; relying on preexisting environment
✅ Configuration loaded successfully
✅ Secrets loaded successfully


## Configuration of HuggingFace caches

In the next cell, we configure HuggingFace cache, so that all the models downloaded from them are persisted locally, even after the workspace is closed. This is a future desired feature for AI Studio and the GenAI addon.


In [9]:
# Initialize HuggingFace Embeddings
embeddings = HuggingFaceEmbeddings()

2025-09-18 12:12:31,346 - INFO - PyTorch version 2.7.1 available.
2025-09-18 12:12:31,753 - INFO - Use pytorch device_name: cuda
2025-09-18 12:12:31,755 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Proxy Configuration

For certain enterprise networks, connecting to external services might require an explicit setup of the proxy configuration. If this is your case, set up the "proxy" field on your config.yaml and the following cell will configure the necessary environment variable.

In [10]:
configure_proxy(config)

# Verify Assets

In [11]:
def log_asset_status(asset_path: str, asset_name: str, success_message: str, failure_message: str) -> None:
    """
    Logs the status of a given asset based on its existence.

    Parameters:
        asset_path (str): File or directory path to check.
        asset_name (str): Name of the asset for logging context.
        success_message (str): Message to log if asset exists.
        failure_message (str): Message to log if asset does not exist.
    """
    if Path(asset_path).exists():
        logger.info(f"{asset_name} is properly configured. {success_message}")
    else:
        logger.info(f"{asset_name} is not properly configured. {failure_message}")

def log_secrets_status(secrets: Dict[str, Any], success_message: str, failure_message: str) -> None:
    """
    Logs the status of secrets based on their existence.

    Parameters:
        secrets (Dict[str, Any]): Secrets retrieved to check if they exist.
        success_message (str): Message to log if secrets exists.
        failure_message (str): Message to log if secrets do not exist.
    """
    if secrets:
        logger.info(f"Project secrets are available. {success_message}")
    else:
        logger.info(f"There are no project secrets found. {failure_message}")


log_asset_status(
    asset_path=CONFIG_PATH,
    asset_name="Config",
    success_message="",
    failure_message="Please check if the configs.yaml was propely connfigured in your project on AI Studio."
)

log_secrets_status(
    secrets=secrets,
    success_message="",
    failure_message="Please check if the secrets were propely connfigured in your project's Secrets Manager."
)

log_asset_status(
    asset_path=LOCAL_MODEL_PATH,
    asset_name="Local Llama model",
    success_message="",
    failure_message="Please create and download the required assets in your project on AI Studio if you want to use local model."
)

2025-09-18 12:12:40 - INFO - Config is properly configured. 
2025-09-18 12:12:40 - INFO - There are no project secrets found. Please check if the secrets were propely connfigured in your project's Secrets Manager.
2025-09-18 12:12:40 - INFO - Local Llama model is properly configured. 


# Data Loading

In this step, we will use the Langchain framework to  extract the content from a local PDF file with the product documentation. Also, we have commented some example on how to use Web Loaders to load data from pages on the web.

In [12]:
# === Verify existence of the data directory ===
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"'data' folder not found at path: {os.path.abspath(DATA_PATH)}")

# === Load PDF document using PyMuPDF ===
file_path = os.path.join(DATA_PATH, "AIStudioDoc.pdf")
pdf_loader = PyMuPDFLoader(file_path)
pdf_data = pdf_loader.load()

# === Optional: Load additional web-based documents ===
# To use a different knowledge base, just change the URLs below

# loader1 = WebBaseLoader("https://www.hp.com/us-en/workstations/ai-studio.html")
# data1 = loader1.load()

# loader2 = WebBaseLoader("https://zdocs.datascience.hp.com/docs/aistudio")
# data2 = loader2.load()

# Creation of Chunks
Here, we split the loaded documents into chunks, so we have smaller and more specific texts to add to our vector database.

In [13]:
# === Initialize text splitter ===
# - chunk_size: Maximum number of characters per text chunk.
# - chunk_overlap: Number of overlapping characters between chunks.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

# === Split the loaded PDF data into smaller text chunks ===
splits = text_splitter.split_documents(pdf_data)

# Retrieval

We transform the texts into embeddings and store them in a vector database. This allows us to perform similarity search, and proper retrieval of documents

In [14]:
%%time

# === Create a vector database from document chunks ===
vectordb = Chroma.from_documents(documents=splits, embedding=embeddings)

# === Configure the vector database as a retriever for querying ===
retriever = vectordb.as_retriever()

2025-09-18 12:12:40,893 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-09-18 12:12:40,999 - ERROR - Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
2025-09-18 12:12:41,008 - ERROR - Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


CPU times: user 744 ms, sys: 187 ms, total: 931 ms
Wall time: 624 ms


# Model Setup

In this notebook, we provide three different options for loading the model:
 * **local**: by loading the Meta Llama 3.1 model with 8B parameters from the asset downloaded on the project
 * **hugging-face-local** by downloading a DeepSeek model from Hugging Face and running locally
 * **hugging-face-cloud** by accessing the Mistral model through Hugging Face cloud API (requires HuggingFace API key saved in Secrets Manager)

This choice can be set in the config.yaml file. The model deployed on the bottom cells of this notebook will load the choice from the config file.

In [15]:
model_source = config["model_source"]

In [16]:
%%time

llm = initialize_llm(model_source, secrets)

CPU times: user 558 ms, sys: 3.65 s, total: 4.2 s
Wall time: 5.49 s


# Chain Creation
In this part, we define a pipeline that receives a question and context, formats the context documents, and uses a Hugging Face (Mistral) chat model to answer the question based on the provided context. The output is then formatted as a string for easy reading.

In [17]:
# === Function to format retrieved documents ===
# Converts a list of Document objects into a single formatted string
def format_docs(docs: List[Document]) -> str:
    return "\n\n".join([d.page_content for d in docs])

In [18]:
# === Define chatbot prompt template ===
# Uses model-specific formatting to prevent hallucination
template = format_rag_chatbot_prompt(model_source)
prompt = ChatPromptTemplate.from_template(template)

# === Create an LLM-powered retrieval chain ===
# - The retriever fetches relevant documents.
# - The documents are formatted using format_docs().
# - The query is passed directly using RunnablePassthrough().
# - The formatted context and query are injected into the prompt.
# - The LLM processes the prompt and the response is parsed into a string.
chain = {
    "context": retriever | format_docs,
    "query": RunnablePassthrough()
} | prompt | llm | StrOutputParser()

# Testing the RAG Chain

Now let's test our RAG chain with some sample questions to verify it's working correctly.

In [19]:
# === Test the RAG chain with sample questions ===
test_questions = [
    "What is AI Studio?",
    "How to create projects in AI Studio?",
    "What are the different workspaces available?",
    "How do I access files on the cloud?"
]

print("🔍 Testing the RAG chain with sample questions:\n")

for i, question in enumerate(test_questions[:2], 1):  # Test first 2 questions
    print(f"Question {i}: {question}")
    try:
        answer = chain.invoke(question)
        print(f"Answer: {answer}\n")
    except Exception as e:
        print(f"Error: {str(e)}\n")
    print("-" * 80)

2025-09-18 12:12:46,875 - ERROR - Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


🔍 Testing the RAG chain with sample questions:

Question 1: What is AI Studio?
Answer: AI Studio is a Data Science platform created by HP.

--------------------------------------------------------------------------------
Question 2: How to create projects in AI Studio?
Answer: To create a project in AI Studio, you can follow these general steps:

1. Click on the 'Start Workspace' button.
2. Name your new workspace.

Once you have done this, you will be able to start working on your project in AI Studio.

--------------------------------------------------------------------------------


In [20]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-09-18 12:13:02 - INFO - ⏱️ Total execution time: 0m 42.18s
2025-09-18 12:13:02 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using [**HP AI Studio**](https://hp.com/ai-studio).